# from https://www.kaggle.com/chmaxx/finetune-vgg16-0-97-with-minimal-effort

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.callbacks import TensorBoard
from keras_tqdm import TQDMNotebookCallback
from keras import backend as K

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions

from datetime import datetime
import os

import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None
pd.options.display.max_rows = 40

Using TensorFlow backend.


In [2]:
vgg16 = VGG16(weights = 'imagenet',include_top=False)

In [3]:
vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [4]:
x = vgg16.get_layer('block5_conv3').output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

model_final = Model(inputs=vgg16.input, outputs=x)

In [5]:
for layer in vgg16.layers:
    layer.trainable = False

model_final.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [7]:
# You need to have these three folders each with two subfolders for the two classes.
train_data_dir = "data/train"
validation_data_dir = "data/valid"
test_data_dir = "data/test"

batch_size = 5 # i achieved good and fast results with this small minibatch size for training
batch_size_val = 5 # if Tensorflow throws a memory error while validating at end of epoch, decrease validation batch size her

# set data augmentation parameters here
datagen = ImageDataGenerator( 
    featurewise_center            = True,
    rescale                       = 1.,
    rotation_range                = 10,
    width_shift_range             = .1,
    height_shift_range            = .1,
    shear_range                   = 0.2,
    zoom_range                    = 0.2,
    horizontal_flip               = True,
    vertical_flip                 = False,
    fill_mode                     = "reflect")

# normalization neccessary for correct image input to VGG16
datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)

# no data augmentation for validation and test set
validgen = ImageDataGenerator(rescale=1., featurewise_center=True)
validgen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)

In [8]:
# 600/450 _ 500/375 _ 400/300 _ 300/225

img_width = 300  # Change image size for training here
img_height = 225 # Change image size for training here

train_gen = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True, 
        #save_to_dir="_augmented_images/", 
        #save_prefix="aug_"
        )

val_gen = validgen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True)

test_gen = validgen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=1,
        class_mode="binary",
        shuffle=False)

train_samples = len(train_gen.filenames)
validation_samples = len(val_gen.filenames)
test_samples = len(test_gen.filenames)

Found 1833 images belonging to 2 classes.
Found 462 images belonging to 2 classes.
Found 1531 images belonging to 1 classes.


In [9]:
now = datetime.now()

# "_tf_logs" is my Tensorboard folder. Change this to your setup if you want to use TB
logdir = "_tf_logs/" + now.strftime("%Y%m%d-%H%M%S") + "/"
tb = TensorBoard(log_dir=logdir)

checkpoint = ModelCheckpoint("weights-1-{epoch:02d}-{val_acc:.2f}.hdf5",
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=False,
                             save_weights_only=True)

epochs=10

# I stopped training automagically with EarlyStopping after 3 consecutive epochs without improvement
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

model_final.fit_generator(train_gen, epochs=epochs, 
                          steps_per_epoch=int(train_samples/batch_size), 
                          validation_data=val_gen, 
                          validation_steps=batch_size_val, 
                          verbose=0, callbacks=[early_stopping, tb, TQDMNotebookCallback(), checkpoint])

Epoch 00008: early stopping



In [10]:
# 600/450 _ 500/375 _ 400/300 _ 300/225

img_width = 600  # Change image size for training here
img_height = 450 # Change image size for training here

train_gen = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True, 
        #save_to_dir="_augmented_images/", 
        #save_prefix="aug_"
        )

val_gen = validgen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True)

test_gen = validgen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=1,
        class_mode="binary",
        shuffle=False)

train_samples = len(train_gen.filenames)
validation_samples = len(val_gen.filenames)
test_samples = len(test_gen.filenames)

Found 1833 images belonging to 2 classes.
Found 462 images belonging to 2 classes.
Found 1531 images belonging to 1 classes.


In [11]:
now = datetime.now()

# "_tf_logs" is my Tensorboard folder. Change this to your setup if you want to use TB
logdir = "_tf_logs/" + now.strftime("%Y%m%d-%H%M%S") + "/"
tb = TensorBoard(log_dir=logdir)

checkpoint = ModelCheckpoint("weights-2-{epoch:02d}-{val_acc:.2f}.hdf5",
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=False,
                             save_weights_only=True)

epochs=10

# I stopped training automagically with EarlyStopping after 3 consecutive epochs without improvement
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

model_final.fit_generator(train_gen, epochs=epochs, 
                          steps_per_epoch=int(train_samples/batch_size), 
                          validation_data=val_gen, 
                          validation_steps=batch_size_val, 
                          verbose=0, callbacks=[early_stopping, tb, TQDMNotebookCallback(), checkpoint])

Epoch 00005: early stopping



In [12]:
for i, layer in enumerate(model_final.layers):
   print(i, layer.name)

for layer in model_final.layers[:15]:
   layer.trainable = False
for layer in model_final.layers[15:]:
   layer.trainable = True

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 global_average_pooling2d_1
19 dense_1
20 dropout_1
21 dense_2


In [13]:
model_final.compile(optimizer=SGD(lr=0.0001, momentum=0.9, nesterov=True),  loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
now = datetime.now()

# "_tf_logs" is my Tensorboard folder. Change this to your setup if you want to use TB
logdir = "_tf_logs/" + now.strftime("%Y%m%d-%H%M%S") + "/"
tb = TensorBoard(log_dir=logdir)

epochs=50

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

checkpoint = ModelCheckpoint("weights-3-{epoch:02d}-{val_acc:.2f}.hdf5",
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=False,
                             save_weights_only=True)

model_final.fit_generator(train_gen, epochs=epochs, 
                          steps_per_epoch=int(train_samples/batch_size), 
                          validation_data=val_gen, 
                          validation_steps=int(validation_samples/batch_size), 
                          verbose=0, callbacks=[early_stopping, tb, TQDMNotebookCallback(), checkpoint])

Epoch 00008: early stopping



In [16]:
preds = model_final.predict_generator(test_gen, 1531)

In [17]:
preds

array([[  1.00000000e+00],
       [  1.00000000e+00],
       [  1.00000000e+00],
       ..., 
       [  4.41600643e-02],
       [  7.95541564e-06],
       [  8.34533479e-03]], dtype=float32)

In [18]:
preds_filenames = test_gen.filenames

In [21]:
preds_filenames = [int(x.replace("unknown/", "").replace(".jpg", "")) for x in preds_filenames]

In [22]:
preds_filenames

[1054,
 639,
 691,
 1244,
 19,
 584,
 113,
 692,
 411,
 813,
 750,
 462,
 124,
 819,
 985,
 838,
 99,
 1044,
 688,
 489,
 261,
 1265,
 508,
 497,
 612,
 1348,
 271,
 1319,
 942,
 892,
 618,
 1288,
 1476,
 491,
 1349,
 244,
 521,
 386,
 909,
 518,
 587,
 199,
 103,
 120,
 1026,
 862,
 1010,
 880,
 1123,
 335,
 370,
 503,
 617,
 231,
 509,
 1367,
 927,
 1037,
 474,
 1383,
 1018,
 163,
 1030,
 356,
 1110,
 764,
 893,
 758,
 658,
 308,
 1119,
 149,
 71,
 1338,
 412,
 1129,
 1284,
 239,
 1260,
 77,
 1421,
 1360,
 1093,
 1187,
 341,
 66,
 1426,
 374,
 490,
 1515,
 1279,
 565,
 1336,
 983,
 1529,
 416,
 1305,
 613,
 1394,
 695,
 1239,
 218,
 109,
 948,
 388,
 863,
 184,
 916,
 1116,
 287,
 286,
 1041,
 535,
 453,
 1247,
 1192,
 311,
 905,
 773,
 171,
 17,
 1153,
 1229,
 476,
 1403,
 908,
 596,
 1004,
 721,
 818,
 1453,
 270,
 414,
 1106,
 1201,
 1330,
 1157,
 316,
 1402,
 1370,
 1084,
 1342,
 976,
 1112,
 502,
 623,
 886,
 92,
 479,
 1356,
 330,
 1361,
 1050,
 904,
 1258,
 578,
 918,
 12,
 68

In [37]:
df_result = pd.DataFrame({'name': preds_filenames, 'invasive': preds[:,0]})
df_result = df_result.sort_values("name")
df_result.index = df_result["name"]
df_result = df_result.drop(["name"], axis=1)

In [38]:
df_result

,invasive
name,
1,1.000000e+00
2,7.468943e-04
3,9.548811e-02
4,2.622996e-03
5,1.000000e+00
6,5.595422e-04
7,5.378333e-02
8,1.000000e+00
9,1.000000e+00


In [39]:
df_result.to_csv("submission_01.csv", encoding="utf8", index=True) #0.98680

In [40]:
model_final.load_weights('./weights-1-02-1.00.hdf5')

In [41]:
preds = model_final.predict_generator(test_gen, 1531)
preds_filenames = test_gen.filenames
preds_filenames = [int(x.replace("unknown/", "").replace(".jpg", "")) for x in preds_filenames]
df_result = pd.DataFrame({'name': preds_filenames, 'invasive': preds[:,0]})
df_result = df_result.sort_values("name")
df_result.index = df_result["name"]
df_result = df_result.drop(["name"], axis=1)
df_result.to_csv("submission_02.csv", encoding="utf8", index=True) #0.51792

In [42]:
model_final.load_weights('./weights-2-01-1.00.hdf5')
preds = model_final.predict_generator(test_gen, 1531)
preds_filenames = test_gen.filenames
preds_filenames = [int(x.replace("unknown/", "").replace(".jpg", "")) for x in preds_filenames]
df_result = pd.DataFrame({'name': preds_filenames, 'invasive': preds[:,0]})
df_result = df_result.sort_values("name")
df_result.index = df_result["name"]
df_result = df_result.drop(["name"], axis=1)
df_result.to_csv("submission_03.csv", encoding="utf8", index=True) #0.52201